<a href="https://colab.research.google.com/github/gauravreddy08/pytorch-vision-transformer/blob/main/vit-replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Replicating **Vision Transformer** using **PyTorch**

<img src="https://raw.githubusercontent.com/gauravreddy08/pytorch-vision-transformer/main/assets/architecture.png" alt="Architecture " width="550">

> **Vision Transformer Paper:** [**An image is worth 16x16 words**](https://arxiv.org/abs/2010.11929)

> **Original Transformer Paper:** [**Attention is all you need**](https://arxiv.org/abs/1706.03762)

In [ ]:
!pip install -q torchinfo wandb

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wandb

import torch
from torch import nn
import torchvision
from torchinfo import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

def set_seed(seed: int = 42):
  torch.manual_seed(42)
  torch.cuda.manual_seed(42)

Device: cpu


In [3]:
# Self Made helper_functions to ease our model development and training process
# Visit : https://github.com/gauravreddy08/learning-pytorch/tree/main/going_modular

!git clone https://github.com/gauravreddy08/learning-pytorch
!mv /content/learning-pytorch/going_modular .
!rm -rf learning-pytorch

Cloning into 'learning-pytorch'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 59 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), 1.77 MiB | 7.34 MiB/s, done.


In [4]:
# Initiating WandB project 
run = wandb.init(project="vit-replication")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Downloading the **dataset**

> **Source:** [`github.com/mrdbourke/pytorch-deep-learning/pizza_steak_sushi.zip`](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi.zip)

In [5]:
import os
import zipfile

from pathlib import Path
import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    data = Path("data/")
    data_path = data /destination
    if(data_path.is_dir()):
      print("[INFO] Data already exists...")
    else:
      print(f"[INFO] Did not find {data_path} directory, creating one...")
      data_path.mkdir(parents=True, exist_ok=True)

      target_file = Path(source).name
      with open(data/target_file, 'wb') as f:
        request = requests.get(source)
        print(f"[INFO] Downloading {target_file} from {source}...")
        f.write(request.content)
      with zipfile.ZipFile(data/target_file, 'r') as zip_ref:
        print(f"[INFO] Unzipping {target_file} data...")
        zip_ref.extractall(data_path)
      if remove_source:
        os.remove(data/target_file)

    return data_path

data_path = download_data("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip", 
                          destination='pizaa_steak_sushi')

train_dir = data_path/'train'
test_dir = data_path/'test'

print(f"Training Directory: {train_dir}")
print(f"Testing Directory: {test_dir}")

[INFO] Did not find data/pizaa_steak_sushi directory, creating one...
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...
Training Directory: data/pizaa_steak_sushi/train
Testing Directory: data/pizaa_steak_sushi/test


## Turning **data** into **dataloaders**